# Full CNN

In [1]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

# 读取数据

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)
y = y.reshape(-1,1)
y = Scaler.fit_transform(y)

# 建立模型

In [ ]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(64,(3,3), activation='relu',padding='same', input_shape=(1000,10,6)))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(64,(3,3),padding='same',activation='relu'))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(128,(3,3),padding='same',activation='relu'))
model_1.add(layers.Flatten())
model_1.add(layers.Dropout(0.5))
#model_1.add(layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=-1)))
#model_1.add(layers.LSTM(units=128, return_sequences=True))
model_1.add(layers.Dense(128,activation='relu'))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(1,activation='relu'))

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(optimizer=optimizers.Adam(lr=0.0001),
               loss='mse',
               metrics=['mae']
               )

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)
model_1.fit(X,y,epochs=200,batch_size=5,callbacks=[early_stopping])

In [ ]:
res = model_1.predict(X)

In [ ]:
plt.plot(Scaler.inverse_transform(res))
plt.plot(Scaler.inverse_transform(y))

In [ ]:
res = Scaler.inverse_transform(res)
res = pd.DataFrame(res)
res.to_csv('E:\\毕业设计\\毕业\\V0\\data\\容量预测结果_全CNN模型.csv',index=False,encoding="utf_8_sig")

In [ ]:
model_1.save('E:\\毕业设计\\毕业\\V0\\data\model\\full_cnn.h5')

# 预测概率分布

In [2]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from scipy.signal import savgol_filter

In [16]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [17]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)

In [18]:
for i in range(304):
    if i < 150:
        y[i] = 0
    elif i < 250:
        y[i] = 1
    else:
        y[i] = 2

In [19]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(64,(3,3), activation='relu',padding='same', input_shape=(1000,10,6)))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(64,(3,3),padding='same',activation='relu'))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(128,(3,3),padding='same',activation='relu'))
model_1.add(layers.Flatten())
model_1.add(layers.Dropout(0.5))
#model_1.add(layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=-1)))
#model_1.add(layers.LSTM(units=128, return_sequences=True))
model_1.add(layers.Dense(128,activation='relu'))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(3,activation='softmax'))

In [20]:
model_1.compile(optimizer=optimizers.Adam(lr=0.0001),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy']
               )

In [21]:
model_1.fit(X,y,epochs=50,batch_size=5)

Epoch 1/50
61/61 [==============================] - 7s 109ms/step - loss: 1.0783 - accuracy: 0.4309
Epoch 2/50
61/61 [==============================] - 8s 127ms/step - loss: 1.0483 - accuracy: 0.4474
Epoch 3/50
61/61 [==============================] - 8s 129ms/step - loss: 1.0444 - accuracy: 0.4934
Epoch 4/50
61/61 [==============================] - 8s 123ms/step - loss: 1.0355 - accuracy: 0.4934
Epoch 5/50
61/61 [==============================] - 8s 124ms/step - loss: 1.0332 - accuracy: 0.4770
Epoch 6/50
61/61 [==============================] - 7s 119ms/step - loss: 1.0213 - accuracy: 0.5132
Epoch 7/50
61/61 [==============================] - 7s 111ms/step - loss: 1.0108 - accuracy: 0.5033
Epoch 8/50
61/61 [==============================] - 7s 113ms/step - loss: 0.9840 - accuracy: 0.5296
Epoch 9/50
61/61 [==============================] - 7s 111ms/step - loss: 1.0260 - accuracy: 0.4704
Epoch 10/50
61/61 [==============================] - 7s 109ms/step - loss: 0.9414 - accuracy: 0.5428

KeyboardInterrupt: 

In [31]:
res = model_1.predict(X)

In [40]:
temp = pd.DataFrame(res)
temp.iloc[:,0] = state_1
temp.iloc[:,1] = state_2
temp.iloc[:,2] = state_3
temp.to_csv('E:\\毕业设计\\毕业\\V0\\data\状态概率预测.csv',index=False,encoding="utf_8_sig")

# CNN+LSTM混合模型

混合模型需要训练好长时间才能效果好，至少要几百个轮次

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.layers import concatenate, LSTM, Conv1D, Conv2D, Flatten, TimeDistributed, Input, Dense, MaxPooling1D, MaxPooling2D, Dropout, Activation
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)
y = y.reshape(-1,1)
y = Scaler.fit_transform(y)

In [ ]:
#X = np.expand_dims(X, axis=-1)
#X.shape

In [ ]:
model_2 = models.Sequential()

In [ ]:
model_2.add(TimeDistributed(Conv1D(64,9, activation='relu',padding='same', input_shape=(1000,10,6))))

In [ ]:
model_2.add(TimeDistributed(MaxPooling1D()))

In [ ]:
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))

In [ ]:
model_2.add(TimeDistributed(MaxPooling1D()))

In [ ]:
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))

In [ ]:
model_2.add(TimeDistributed(Flatten())) 

In [ ]:
model_2.add(Dropout(0.5)) 

In [ ]:
model_2.add(LSTM(units=128, return_sequences=True))

In [ ]:
model_2.add(LSTM(units=50, return_sequences=False))

In [ ]:
#model.add(Dense(128,activation='relu'))
#model.add(Dense(64,activation='relu'))
model_2.add(Dense(1,activation='relu'))

In [ ]:
model_2.compile(optimizer=optimizers.Adam(lr=0.0001),
               loss='mse',
               metrics=['mae'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)
model_2.fit(X,y,epochs=1,batch_size=5,callbacks=[early_stopping])

In [ ]:
res = model.predict(X)

In [ ]:
plt.plot(Scaler.inverse_transform(res))
plt.plot(Scaler.inverse_transform(y))

In [ ]:
res = Scaler.inverse_transform(res)
res = pd.DataFrame(res)
res.to_csv('E:\\毕业设计\\毕业\\V0\\data\\容量预测结果_混合模型.csv',index=False,encoding="utf_8_sig")

In [ ]:
model_2.save('E:\\毕业设计\\毕业\\V0\\data\model\\cnn+lstm.h5')

# 贝叶斯网络推断

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.factors.discrete import TabularCPD
import pandas as pd
from pgmpy.inference import VariableElimination
from graphviz import Digraph

In [ ]:
battery_model = BayesianModel([('Cell_3', 'Road_2'), 
                              ('Cell_4', 'Road_2'),
                              ('Road_1', 'Battery_Pack'),
                              ('Road_2', 'Battery_Pack')])

In [ ]:
def showBN(model,save=False):
    '''传入BayesianModel对象，调用graphviz绘制结构图，jupyter中可直接显示'''
    node_attr = dict(
     style='filled',
     shape='box',
     align='left',
     fontsize='12',
     ranksep='0.1',
     height='0.2'
    )
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    edges=model.edges()
    for a,b in edges:
        dot.edge(a,b)
    if save:
        dot.view(cleanup=True)
    return dot
showBN(battery_model) 

In [ ]:
#model = BayesianModel([('Cell_3', 'Road_2'), ('Cell_4', 'Road_2')])
#model = BayesianModel([('A', 'C'), ('B', 'C')])

In [ ]:
data = pd.read_csv('E:\\毕业设计\\毕业\\V0\data\\电池组状态数据.csv')
#data = pd.DataFrame(data={'A': [0, 0, 1], 'B': [0, 1, 0], 'C': [1, 1, 0]})

In [ ]:
estimator = BayesianEstimator(model, data)

In [ ]:
cpd_C = estimator.estimate_cpd('Road_2')#, prior_type="dirichlet", pseudo_counts=[1,2])

In [ ]:
print(cpd_C)

In [ ]:
cpd_cell3 = TabularCPD(variable='Cell_3', variable_card=3,
                      values=[[0.6], [0.2], [0.2]])
cpd_cell4 = TabularCPD(variable='Cell_4', variable_card=3,
                       values=[[0.6], [0.2], [0.2]])
cpd_road1 = TabularCPD(variable='Road_1', variable_card=3,
                       values=[[0.9],[0.0],[0.1]])
cpd_road2 = TabularCPD(variable='Road_2', variable_card=3,
                        values=[[0.98, 0.3, 0.2, 0.3, 0.0037, 0.0333, 0.1, 0.0208, 0.0088],
                                [0.01, 0.6, 0.4, 0.6, 0.9370, 0.9333, 0.6, 0.3333, 0.1193],
                                [0.01, 0.1, 0.4, 0.1, 0.0593, 0.0333, 0.3, 0.6458, 0.8719]],
                        evidence=['Cell_3', 'Cell_4'],
                        evidence_card=[3,3])
cpd_pack = TabularCPD(variable='Battery_Pack', variable_card=3,
                      values=[[0.9188, 0.6465, 0.3333, 0.3333, 0.0037, 0.0333, 0.3333, 0.0208, 0.0088],
                              [0.0784, 0.3434, 0.3333, 0.3333, 0.9370, 0.9333, 0.3333, 0.3333, 0.0193],
                              [0.0028, 0.0101, 0.3333, 0.3333, 0.0593, 0.0333, 0.3333, 0.6458, 0.9719]],
                      evidence=['Road_1', 'Road_2'], evidence_card=[3,3])

In [ ]:
battery_model.add_cpds(cpd_cell3, cpd_cell4, cpd_road1, cpd_road2, cpd_pack)

In [ ]:
battery_model.check_model()

In [ ]:
battery_model.get_independencies()

In [ ]:
model_infer = VariableElimination(battery_model)
q = model_infer.query(variables=['Road_2'], evidence={})

In [ ]:
print(q)

In [ ]:
data = pd.read_csv('E:\\毕业设计\\毕业\\V0\data\\电池组状态数据.csv')

In [ ]:
predict_data = data.drop(columns=['Battery_Pack'],axis=1)

In [ ]:
predict_data

In [ ]:
y_pred = battery_model.predict(predict_data)

In [ ]:
y_pred

In [ ]:
print((y_pred['Battery_Pack'] == data['Battery_Pack']).sum()/len(data))